# Light Beads Microscopy: Exploration 


In [9]:
import sys
import os
from pathlib import Path
import numpy as np
import cv2
sys.path.append('../../util/')  # TODO: Take this out when we upload to pypi
sys.path.append('../../exclude/')  # TODO: Take this out when we upload to pypi
import scanreader
import util

import bokeh.plotting as bpl
import holoviews as hv
from IPython import get_ipython
import logging
import matplotlib.pyplot as plt

try:
    cv2.setNumThreads(0)
except():
    pass

try:
    if __IPYTHON__:
        get_ipython().run_line_magic('load_ext', 'autoreload')
        get_ipython().run_line_magic('autoreload', '2')
except NameError:
    pass

import caiman as cm

bpl.output_notebook()
hv.notebook_extension('bokeh')
%matplotlib inline

datapath = Path('/data2/fpo/data/')                 # string pointing to directory containing your data
tiffs = [x for x in datapath.glob('*.tif')]         # this accumulates a list of every filepath which contains a .tif file
reader = scanreader.read_scan(str(tiffs[0]), join_contiguous=True, lbm=True, x_cut=(5,5))
data = reader[0]

In [6]:
def linear_stitch(left, right, expected_delta_x, minimum_overlap=10):
    """Compute offsets to stitch two side-by-side volumes via cross-correlation.

    Subpixel shifts are calculated per depth and the median is returned.

    Arguments:
    :param left: Slice (height x width) to be stitched in the left.
    :param right: Slice (height x width) to be stitched in the right.
    :param float expected_delta_x: Expected distance between center of left to right.
    :param float minimum_overlap: Minimum overlap after stitching.

    :returns: (delta_y, delta_x). Distance between center of right ROI and left ROI after
        stitching (right_center - left_center)
    """
    # Get some params
    right_height, right_width = right.shape
    left_height, left_width = left.shape
    expected_overlap = left_width / 2 + right_width / 2 - expected_delta_x

    # Drop some rows, columns to avoid artifacts
    skip_columns = int(round(0.05 * expected_overlap))  # 5% of expected overlap
    skip_rows = int(
        round(0.05 * min(right_height, left_height))
    )  # 5% of top and bottom rows
    left = left[skip_rows:-skip_rows, :-skip_columns]
    right = right[skip_rows:-skip_rows, skip_columns:]
    expected_overlap = int(round(expected_overlap - 2 * skip_columns))

    # Cut strips of expected overlap (plus some padding on the sides)
    min_height = min(left.shape[0], right.shape[0])
    pad_pixels = int(round(expected_overlap * 0.5 - minimum_overlap))
    left_strip = np.zeros([min_height, expected_overlap + 2 * pad_pixels])
    right_strip = np.zeros([min_height, expected_overlap + 2 * pad_pixels])
    left_strip[:, : expected_overlap + pad_pixels] = left[
        :min_height, -(expected_overlap + pad_pixels) :
    ]
    right_strip[:, -(expected_overlap + pad_pixels) :] = right[
        :min_height, : expected_overlap + pad_pixels
    ]

    # Compute best match
    y_shifts, x_shifts = util.compute_motion_shifts(right_strip, left_strip, in_place=False)
    y_shift, x_shift = y_shifts[0], x_shifts[0]

    # Compute right_center minus left_center
    right_ycenter = right_height / 2 - y_shift  # original + offset
    right_xcenter = (
        right_width / 2 + (left_width - expected_overlap - 2 * skip_columns) - x_shift
    )  # original + repositioning + offset
    delta_y = right_ycenter - left_height / 2  # negative to change direction of y axis
    delta_x = right_xcenter - left_width / 2

    return delta_y, delta_x

In [7]:
x = linear_stitch(roi_1[:, :, 5, 5], roi_2[:, :, 5, 5], 0.5)

In [ ]:
slice_plane = roi_1[:,:,5,:]
phase_angle =  util.compute_raster_phase(slice_plane[:,:, 400], reader.temporal_fill_fraction)
corrected_li = util.correct_raster(slice_plane, phase_angle, reader.temporal_fill_fraction)